# Шагайн дүрсийг таних модель сургах

Шагайн хэлбэр буюу морь, тэмээ, хонь, ямаа гэсэн 4 төрлийн дүрсийг зурагнаас ялгаж таних моделийг үүсгэнэ.
Моделийг үүсгэхдээ RetineNet ([https://arxiv.org/abs/1708.02002](https://arxiv.org/abs/1708.02002)) архитектурыг ашигласан.
![](images/RetinaNet.png)
RetinaNet нь ResNet архитектур дээр Feature Pyramid Network-ийг ашиглаж (a) олон төрлийн хэмжээтэй convolutional feature pyramid үүсгэдэг (b). Энэ RetinaNet-ийн сууринд хоёр дэд сүлжээ холбогддог. Нэг нь anchor box-ыг ангилдаг (c), нөгөө нь anchor box-ыг объектын хайрцаг болгодог (d). Энэ сүлжээ нь нэг үет илрүүлэгч (one-stage detector) ашигладаг ба хоёр үет илрүүлэгчийг (two-stage detector) (жишээ нь Faster-RCNN) бодвол илүү хурдан ажилладаг боловч нарийвчлал багатай. Харин loss тооцохдоо focal loss функцийг ашигласнаар нэг үет болон хоёр үет хоорондын нарийвчлалын ялгааг алга болгож, илүү хурдан бөгөөд нарийвчлал сайтай ажиллах боломжийг олгодог.

## Шаардлагатай сангуудыг унших

Ашигласан үндсэн сангууд:

- `Tensorflow` - Keras-ийн backend

- `Keras`

- `Keras RetinaNet` - RetinaNet-ийг Keras дээр хэрэгжүүлсэн сан

In [ ]:
import os
import keras
import tensorflow as tf
from keras_retinanet import models
from keras_retinanet import layers
from keras_retinanet import losses
from keras_retinanet.models.retinanet import retinanet_bbox
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.utils.anchors import make_shapes_callback
from keras_retinanet.utils.model import freeze as freeze_model
from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.callbacks.eval import Evaluate

## Шаардлагатай параметрууд

- `annotations_path` - `images/hcsg_01.jpg,126,131,159,166,Camel` хэлбэрийн өгөгдөл агуулсан сургалтын файл
- `classes_path` - ангилал, түүнд харгалзах id `horse,0`
- `validations_path` - шалгах өгөгдлийн файл ба сургалтын өгөгдөлтэй адилхан бүтэцтэй файл
- `evaluation` - шалгах эсэх
- `snapshot` - үүссэн моделийг үргэлжлүүлж сурах бол замыг нь зааж өгнө
- `snapshots` - давталт (epoch) болгонд моделийг хадгалах эсэх
- `snapshot_path` - моделийг хадгалах байрлал

In [ ]:
backbone = 'resnet50'
# Annotation файлууд
annotations_path = 'ankle_bones_dataset/annotations.csv'
classes_path = 'ankle_bones_dataset/classes.csv'
validations_path = 'ankle_bones_dataset/validations.csv'
evaluation = True
# Хэрэв сургалтаа үргэлжлүүлэх бол snapshot-ын утганд сүүлийн сургалтын файлыг зааж өгнө
snapshot = None
# Snapshot хадгалах эсэх, хадгалах хавтас
snapshots = True
snapshot_path = './snapshots'

tensorboard_dir = './logs'

weights = None
imagenet_weights = True
batch_size = 1
image_min_side = 225
image_max_side = 300

Backbone модель ResNet50-р үүсгэх

In [ ]:
backbone_model = models.backbone('resnet50')
freeze_backbone = False
common_args = {
    'batch_size'       : batch_size,
    'image_min_side'   : image_min_side,
    'image_max_side'   : image_max_side,
    'preprocess_image' : backbone_model.preprocess_image,
}

## Сургалт болон шалгалтын өгөгдлийн унших

In [ ]:
# Сургалтын болон шалгах өгөгдлөө унших
train_generator = CSVGenerator(annotations_path, classes_path, **common_args)
if validations_path != '':
    validation_generator = CSVGenerator(validations_path, classes_path, **common_args)
else:
    validation_generator = None

## Модель үүсгэх, унших

Өмнө сургасан моделийг ашиглан үргэлжлүүлэн сургах бол тухайн моделийг уншина. Хэрэв шинээр сургах бол шинэ модель үүсгэнэ.

In [ ]:
# Model-оо үүсгэх эсвэл өмнө нь үүссэн model-г унших
if snapshot is not None:
    print('Loading model, this may take a second...')
    model = models.load_model(snapshot, backbone_name=backbone)
    training_model = model
    prediction_model = retinanet_bbox(model=model)
else:
    weights = weights
    # default to imagenet if nothing else is specified
    if weights is None and imagenet_weights:
        weights = backbone_model.download_imagenet()

    print('Creating model, this may take a second...')
    modifier = freeze_model if freeze_backbone else None
    
    model = backbone_model.retinanet(train_generator.num_classes(), modifier=modifier)
    if weights is not None:
        model.load_weights(weights, by_name=True, skip_mismatch=True)
    training_model = model

    # make prediction model
    prediction_model = retinanet_bbox(model=model)

    # compile model
    training_model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            'classification': losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )

# model-н тухай хураангуй
print(model.summary())

# this lets the generator compute backbone layer shapes using the actual backbone model
if 'vgg' in backbone or 'densenet' in backbone:
    train_generator.compute_shapes = make_shapes_callback(model)
    if validation_generator:
        validation_generator.compute_shapes = train_generator.compute_shapes

### Callbacks

In [ ]:
callbacks = []

tensorboard_callback = None

if tensorboard_dir:
    tensorboard_callback = keras.callbacks.TensorBoard(
        log_dir                = tensorboard_dir,
        histogram_freq         = 0,
        batch_size             = batch_size,
        write_graph            = True,
        write_grads            = False,
        write_images           = False,
        embeddings_freq        = 0,
        embeddings_layer_names = None,
        embeddings_metadata    = None
    )
    callbacks.append(tensorboard_callback)

if evaluation and validation_generator:
    evaluation = Evaluate(validation_generator, tensorboard=tensorboard_callback)
    evaluation = RedirectModel(evaluation, prediction_model)
    callbacks.append(evaluation)

# model хадгалах
if snapshots:
    # хадгалах хавтсыг үүсгэх
    try:
        os.makedirs(snapshot_path)
    except OSError:
        if not os.path.isdir(snapshot_path):
            raise
    checkpoint = keras.callbacks.ModelCheckpoint(
        os.path.join(
            snapshot_path,
            '{backbone}_{dataset_type}_{{epoch:02d}}.h5'.format(backbone=backbone, dataset_type='csv')
        ),
        verbose=1,
        # save_best_only=True,
        # monitor="mAP",
        # mode='max'
    )
    checkpoint = RedirectModel(checkpoint, model)
    callbacks.append(checkpoint)

callbacks.append(keras.callbacks.ReduceLROnPlateau(
    monitor  = 'loss',
    factor   = 0.1,
    patience = 2,
    verbose  = 1,
    mode     = 'auto',
    epsilon  = 0.0001,
    cooldown = 0,
    min_lr   = 0
))

## Сургалтыг эхлүүлэх

In [ ]:
epochs = 40
steps = 1000

In [ ]:
# Сургалт эхлүүлэх
training_model.fit_generator(
    generator=train_generator,
    steps_per_epoch=steps,
    epochs=epochs,
    verbose=1,
    callbacks=callbacks,
)